In [4]:
import glob
import 

cTDaR_t00000.jpg  cTDaR_t00457.JPG  cTDaR_t10000.jpg  cTDaR_t10300.jpg
cTDaR_t00000.xml  cTDaR_t00457.xml  cTDaR_t10000.xml  cTDaR_t10300.xml
cTDaR_t00001.jpg  cTDaR_t00458.JPG  cTDaR_t10001.jpg  cTDaR_t10301.jpg
cTDaR_t00001.xml  cTDaR_t00458.xml  cTDaR_t10001.xml  cTDaR_t10301.xml
cTDaR_t00002.jpg  cTDaR_t00459.JPG  cTDaR_t10002.jpg  cTDaR_t10302.jpg
cTDaR_t00002.xml  cTDaR_t00459.xml  cTDaR_t10002.xml  cTDaR_t10302.xml
cTDaR_t00003.jpg  cTDaR_t00460.JPG  cTDaR_t10003.jpg  cTDaR_t10303.jpg
cTDaR_t00003.xml  cTDaR_t00460.xml  cTDaR_t10003.xml  cTDaR_t10303.xml
cTDaR_t00004.jpg  cTDaR_t00461.JPG  cTDaR_t10004.jpg  cTDaR_t10304.jpg
cTDaR_t00004.xml  cTDaR_t00461.xml  cTDaR_t10004.xml  cTDaR_t10304.xml
cTDaR_t00005.jpg  cTDaR_t00462.JPG  cTDaR_t10005.jpg  cTDaR_t10305.jpg
cTDaR_t00005.xml  cTDaR_t00462.xml  cTDaR_t10005.xml  cTDaR_t10305.xml
cTDaR_t00006.jpg  cTDaR_t00463.JPG  cTDaR_t10006.jpg  cTDaR_t10306.jpg
cTDaR_t00006.xml  cTDaR_t00463.xml  cTDaR_t10006.xml  cTDaR_t10306.xml
cTDaR_